In [18]:
import pandas as pd
import numpy as np
from functools import reduce
from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import KNNImputer
from patsy import dmatrix
import numpy as np
import doubleml as dml
from doubleml.datasets import make_iivm_data
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
import math


des_data = pd.read_stata(r"/Users/yasmineouattara/Documents/GitHub/ML_for_Econ_Final_Project/OHIE_Data/oregonhie_descriptive_vars.dta")
ed_data = pd.read_stata(r"/Users/yasmineouattara/Documents/GitHub/ML_for_Econ_Final_Project/OHIE_Data/oregonhie_ed_vars.dta")
inperson_data = pd.read_stata(r"/Users/yasmineouattara/Documents/GitHub/ML_for_Econ_Final_Project/OHIE_Data/oregonhie_inperson_vars.dta")
patterns_data = pd.read_stata(r"/Users/yasmineouattara/Documents/GitHub/ML_for_Econ_Final_Project/OHIE_Data/oregonhie_patterns_vars.dta")
stateprograms_data = pd.read_stata(r"/Users/yasmineouattara/Documents/GitHub/ML_for_Econ_Final_Project/OHIE_Data/oregonhie_stateprograms_vars.dta")
survey0_data = pd.read_stata(r"/Users/yasmineouattara/Documents/GitHub/ML_for_Econ_Final_Project/OHIE_Data/oregonhie_survey0m_vars.dta")
survey6_data = pd.read_stata(r"/Users/yasmineouattara/Documents/GitHub/ML_for_Econ_Final_Project/OHIE_Data/oregonhie_survey6m_vars.dta")
survey12_data = pd.read_stata(r"/Users/yasmineouattara/Documents/GitHub/ML_for_Econ_Final_Project/OHIE_Data/oregonhie_survey12m_vars.dta")

In [19]:
des_data = des_data.apply(lambda x: x.cat.codes if x.dtype.name == 'category' else x)
des_data.replace(-1, np.nan, inplace=True)
des_subdata = des_data[['person_id', 'household_id', 'treatment', 'draw_lottery', 'applied_app', 'approved_app', 
                        'numhh_list', 'birthyear_list', 'have_phone_list', 'english_list', 'female_list', 
                        'first_day_list', 'last_day_list', 'pobox_list', 'self_list', 'week_list', 'zip_msa_list']]
des_subdata.loc[:,"numhh_list"] = des_subdata["numhh_list"] + 1
des_subdata

,person_id,household_id,treatment,draw_lottery,applied_app,approved_app,numhh_list,birthyear_list,have_phone_list,english_list,female_list,first_day_list,last_day_list,pobox_list,self_list,week_list,zip_msa_list
0,1.0,100001.0,1,6,1.0,0.0,1,1978,1,1,0.0,0,0,1,1,1,1.0
1,2.0,100002.0,1,5,0.0,0.0,1,1984,1,1,1.0,0,0,0,1,2,1.0
2,3.0,100003.0,0,1,NaN,NaN,1,1971,1,1,1.0,0,0,0,1,2,1.0
3,4.0,100004.0,0,7,NaN,NaN,1,1955,1,1,1.0,0,0,0,1,0,1.0
4,5.0,100005.0,1,6,0.0,0.0,1,1969,1,0,1.0,0,0,0,1,1,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74917,74918.0,174918.0,0,5,NaN,NaN,1,1955,1,1,0.0,0,0,0,1,1,1.0
74918,74919.0,174919.0,1,5,1.0,1.0,1,1979,0,1,1.0,0,0,0,1,2,0.0
74919,74920.0,174920.0,0,5,NaN,NaN,1,1965,1,1,1.0,0,0,0,1,0,1.0
74920,74921.0,174921.0,1,0,1.0,1.0,1,1948,1,1,1.0,0,0,0,1,3,0.0


In [22]:
# State
stateprograms_data = stateprograms_data.apply(lambda x: x.cat.codes if x.dtype.name == 'category' else x)
stateprograms_data.replace(-1, np.nan, inplace=True)

# Education
ed_data = ed_data.apply(lambda x: x.cat.codes if x.dtype.name == 'category' else x)
ed_data.replace(-1, np.nan, inplace=True)

# In Person
inperson_data = inperson_data.apply(lambda x: x.cat.codes if x.dtype.name == 'category' else x)
inperson_data.replace(-1, np.nan, inplace=True)

# Patterns
patterns_data = patterns_data.apply(lambda x: x.cat.codes if x.dtype.name == 'category' else x)
patterns_data.replace(-1, np.nan, inplace=True)

# Survey 0
survey0_data = survey0_data.apply(lambda x: x.cat.codes if x.dtype.name == 'category' else x)
survey0_data.replace(-1, np.nan, inplace=True)

# Survey 6
survey6_data = survey6_data.apply(lambda x: x.cat.codes if x.dtype.name == 'category' else x)
survey6_data.replace(-1, np.nan, inplace=True)

# Survey 12
survey12_data = survey12_data.apply(lambda x: x.cat.codes if x.dtype.name == 'category' else x)
survey12_data.replace(-1, np.nan, inplace=True)


dfs = [des_subdata, stateprograms_data, ed_data, inperson_data, patterns_data, survey0_data, survey6_data, survey12_data]

merged = reduce(lambda left, right: pd.merge(left, right, on='person_id', how='outer'), dfs)

merged.describe()

,person_id,household_id_x,treatment_x,draw_lottery_x,applied_app,approved_app,numhh_list_x,birthyear_list_x,have_phone_list_x,english_list_x,...,live_alone_12m,live_parents_12m,live_relatives_12m,live_friends_12m,live_other_12m,hhsize_12m,num19_12m,needmet_med_cor_12m,needmet_rx_cor_12m,needmet_dent_cor_12m
count,74922.000000,74922.000000,74922.000000,74922.000000,29799.000000,29799.000000,74922.000000,74922.000000,74922.000000,74922.000000,...,23775.000000,23777.000000,23777.000000,23777.000000,23776.000000,22518.000000,2.272100e+04,22940.000000,22860.000000,23172.000000
mean,37461.500000,140311.609375,0.398201,3.862404,0.608175,0.291889,1.234270,1968.076987,0.864205,0.914044,...,0.167739,0.103167,0.231779,0.103041,0.084918,10.330225,7.102876e+07,0.619137,0.719641,0.410236
min,1.000000,100001.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1945.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000e+00,0.000000,0.000000,0.000000
25%,18731.250000,122288.250000,0.000000,2.000000,0.000000,0.000000,1.000000,1958.000000,1.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000e+00,0.000000,0.000000,0.000000
50%,37461.500000,141644.500000,0.000000,4.000000,1.000000,0.000000,1.000000,1968.000000,1.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000,0.000000e+00,1.000000,1.000000,0.000000
75%,56191.750000,159051.750000,1.000000,6.000000,1.000000,1.000000,1.000000,1979.000000,1.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,4.000000,2.000000e+00,1.000000,1.000000,1.000000
max,74922.000000,174922.000000,1.000000,7.000000,1.000000,1.000000,3.000000,1988.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,163559.000000,1.613841e+12,1.000000,1.000000,1.000000
std,21628.263672,21441.220703,0.489531,2.218033,0.488166,0.454639,0.428495,12.202943,0.342573,0.280301,...,0.373642,0.304183,0.421977,0.304018,0.278765,1089.963867,1.070649e+10,0.485610,0.449184,0.491887
